In [1]:
import numpy as np
import plotly.graph_objects as go
import quantfreedom as qf
from plotly.subplots import make_subplots

from quantfreedom.plotting.simple_plots import plot_results_candles_and_chart

In [80]:
def testing(
    seed=None,
    price_range_start=None,
    price_range_end=None,
    rsi_window=14,
    lookback=20,
):
    price_data = qf.generate_candles(
        number_of_candles=200,
        plot_candles=False,
        seed=seed,
    )
    rsi_data = qf.from_talib(
        func_name="rsi",
        price_data=price_data,
        timeperiod=rsi_window,
        plot_results=False,
        # price="low",
    )
    df = price_data.droplevel(level=0, axis=1)
    df["rsi"] = rsi_data["QuantFreedom"][rsi_window]
    price_open_values = df.open.values
    price_high_values = df.high.values
    price_low_values = df.low.values
    price_close_values = df.close.values

    piv_low = price_low_values[0]
    piv_low_graph = np.full_like(price_low_values, np.nan)

    rsi_values = df.rsi.values
    divergence_index = np.full_like(price_low_values, -1, dtype=np.int_)
    rsi_line_graph = np.full_like(price_low_values, np.nan)

    candle_line_graph = np.full_like(price_low_values, np.nan)

    plot_index = df.index
    line_counter = 0

    if price_range_start is not None:
        for x in range(1, price_low_values.size):
            temp_candle_low = price_low_values[x]
            if temp_candle_low < price_range_start and temp_candle_low > price_range_end:
                if (
                    temp_candle_low < piv_low
                    and price_close_values[x] < price_open_values[x]
                ):
                    for lb in range(1, lookback):
                        lb_index = x - lb
                        if (
                            rsi_values[x] > rsi_values[lb_index]
                            and price_close_values[lb_index] < price_open_values[lb_index]
                        ):
                            candle_line_graph[line_counter + 1] = (
                                price_low_values[lb_index] - 2
                            )
                            candle_line_graph[line_counter] = temp_candle_low - 2

                            rsi_line_graph[line_counter + 1] = rsi_values[lb_index] - 2
                            rsi_line_graph[line_counter] = rsi_values[x] - 2

                            divergence_index[line_counter + 1] = lb_index
                            divergence_index[line_counter] = x

                            line_counter += 2

                            break
                    piv_low = temp_candle_low
                    piv_low_graph[x] = temp_candle_low - 2
                else:
                    pass
    else:
        for x in range(1, price_low_values.size):
            temp_candle_low = price_low_values[x]
            if (
                temp_candle_low < piv_low
                and price_close_values[x] < price_open_values[x]
            ):
                for lb in range(1, lookback):
                    lb_index = x - lb
                    if lb_index > 0:
                        if (
                            rsi_values[x] > rsi_values[lb_index]
                            and price_close_values[lb_index] < price_open_values[lb_index]
                        ):
                            candle_line_graph[line_counter + 1] = (
                                price_low_values[lb_index] - 2
                            )
                            candle_line_graph[line_counter] = temp_candle_low - 2

                            rsi_line_graph[line_counter + 1] = rsi_values[lb_index] - 2
                            rsi_line_graph[line_counter] = rsi_values[x] - 2

                            divergence_index[line_counter + 1] = lb_index
                            divergence_index[line_counter] = x

                            line_counter += 2

                            break
                    else:
                        break
                piv_low = temp_candle_low
                piv_low_graph[x] = temp_candle_low - 2
            else:
                pass
        # keep some type of counter that checks if we have gone 20 candles without a new low
    rsi_line_graph = rsi_line_graph[:line_counter]
    candle_line_graph = candle_line_graph[:line_counter]
    divergence_index = divergence_index[:line_counter]
    divergence_index = plot_index[divergence_index.tolist()]
    fig = make_subplots(
        rows=2,
        cols=1,
        row_heights=[0.7, 0.3],
        shared_xaxes=True,
        vertical_spacing=0.02,
    )
    fig.append_trace(
        go.Candlestick(
            x=plot_index,
            open=df["open"],
            high=df["high"],
            low=df["low"],
            close=df["close"],
        ),
        row=1,
        col=1,
    )

    fig.add_scatter(
        x=plot_index,
        y=piv_low_graph,
        mode="markers",
        marker=dict(size=4, color="MediumPurple"),
        name="candle pivot",
        row=1,
        col=1,
    )
    if price_range_start is not None:
        fig.add_hrect(
            y0=price_range_start,
            y1=price_range_end,
            line_width=2,
            fillcolor="green",
            line_color='white',
            opacity=0.2,
            annotation_text="Price Range",
            annotation_position="top left",
            row=1,
            col=1,
        )

    fig.append_trace(
        go.Scatter(
            x=plot_index,
            y=rsi_values,
            name="RSI",
        ),
        row=2,
        col=1,
    )

    start = 0
    end = 2
    for x in range(rsi_line_graph.size):
        fig.add_scatter(
            x=divergence_index[start:end],
            y=candle_line_graph[start:end],
            mode="markers+lines",
            marker=dict(size=10, symbol="arrow-up"),
            name="Candle Line Div",
            row=1,
            col=1,
            showlegend=False,
        )
        fig.add_scatter(
            x=divergence_index[start:end],
            y=rsi_line_graph[start:end],
            mode="markers+lines",
            marker=dict(size=10, symbol="arrow-up"),
            name="RSI div",
            row=2,
            col=1,
            showlegend=False,
        )
        start = end
        end += 2
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        height=600,
    )
    fig.show()

In [100]:
testing(
    # 18,
    # price_range_start=29930,
    # price_range_end=29900,
    lookback=10,
)

In [68]:
testing(
    18,
    # price_range_start=29930,
    # price_range_end=29900,
)